In [1]:
import numpy as np
from matplotlib import pyplot as plt
import quaternion
import pandas as pd
def phiRL_2_thetabeta(R, L):
    r = np.exp(1j*(R + np.deg2rad(17.0))) 
    l = np.exp(1j*(L - np.deg2rad(17.0)))
    theta = np.angle(r / l)
    for i in range(len(theta)):
        if (theta[i] < 0):
            theta[i] += 2 * np.pi
    theta = theta * 0.5
    beta = np.angle(l) + theta
    return theta, beta
def phiRL_2_thetabeta_vel(R, L, f = 1000):
    r = np.diff(R, axis=0, prepend=np.array([0])) * f
    l = np.diff(L, axis=0, prepend=np.array([0])) * f
    beta = (r + l) * 0.5
    theta = (r - l) * 0.5
    return theta, beta
def mod2pi(arr):
    arr[arr < 0] += np.pi * 2
    arr = np.fmod(arr, np.pi * 2)
    return arr

# "Cx" << "," << "Cy" << "," << "Cz" << "," << 
# "lf.x" << "," << "lf.y" << "," << "lf.z" << "," << 
# "rf.x" << "," << "rf.y" << "," << "rf.z" << "," << 
# "rh.x" << "," << "rh.y" << "," << "rh.z" << "," << 
# "lh.x" << "," << "lh.y" << "," << "lh.z" << "," << 
# "vel.x" << "," << "vel.y" << "," << "vel.z" << "," << 
# "q.w" << "," << "q.x" << "," << "q.y" << "," << "q.z" << "," << 
# "ang_vel.x" << "," << "ang_vel.y" << "," << "ang_vel.z" << "," << 
# "accel.x" << "," << "accel.y" << "," << "accel.z" << "," << 
# "lf.dst" << "," << "rf.dst" << "," << "rh.dst" << "," << "lh.dst" << "," <<
# "lf.r" << "," << "lf.l" << "," << 
# "rf.r" << "," << "rf.l" << "," << 
# "rh.r" << "," << "rh.l" << "," <<
# "lh.r" << "," << "lh.l"

In [3]:
filename = 'controlled'
data = pd.read_csv('data/'+filename+'.csv')[1000:-1]
p = np.array(data[['Cx', 'Cy', 'Cz']])
q = np.array(data[['q.w', 'q.x', 'q.y', 'q.z']])
rots = quaternion.as_rotation_matrix(quaternion.from_float_array(q))#  * np.array([[1, 0, 0], [0, -1, 0], [0, 0, -1]])
# rpy = quaternion.as_euler_angles(quaternion.from_float_array(q))
v = np.array(data[['vel.x', 'vel.y', 'vel.z']])
a = np.array(data[['accel.x', 'accel.y', 'accel.z']])
# a += np.random.normal(0, 0.1, a.shape) + 0.01
w = np.array(data[['ang_vel.x', 'ang_vel.y', 'ang_vel.z']])
# w += np.random.normal(0, 0.01, w.shape) + 0.001
# c = np.array(data[['lf.c', 'rf.c', 'rh.c', 'lh.c']])
# c[c < 0] = 0
dst = np.array(data[["lf.dst", "rf.dst", "rh.dst", "lh.dst"]])
lf_theta, lf_beta = phiRL_2_thetabeta(np.array(data['lf.r']), np.array(data['lf.l']))
lf_theta_d, lf_beta_d = phiRL_2_thetabeta_vel(np.array(data['lf.r']), np.array(data['lf.l']))
rf_theta, rf_beta = phiRL_2_thetabeta(np.array(data['rf.l']), np.array(data['rf.r']))
rf_theta_d, rf_beta_d = phiRL_2_thetabeta_vel(np.array(data['rf.l']), np.array(data['rf.r']))
rh_theta, rh_beta = phiRL_2_thetabeta(np.array(data['rh.l']), np.array(data['rh.r']))
rh_theta_d, rh_beta_d = phiRL_2_thetabeta_vel(np.array(data['rh.l']), np.array(data['rh.r']))
lh_theta, lh_beta = phiRL_2_thetabeta(np.array(data['lh.r']), np.array(data['lh.l']))
lh_theta_d, lh_beta_d = phiRL_2_thetabeta_vel(np.array(data['lh.r']), np.array(data['lh.l']))
contacts = np.array([np.isnan(np.array(data['lf.z'])), 
           np.isnan(np.array(data['rf.z'])), 
           np.isnan(np.array(data['rh.z'])), 
           np.isnan(np.array(data['lh.z']))]).T
size = a.shape[0]
for i, rot in enumerate(rots):
    v[i] = v[i]
    a[i] = a[i] - rot.transpose() @ np.array([0, 0, 9.81])
columns_name = ['C.x', 'C.y', 'C.z', 'v.x', 'v.y', 'v.z', 'a.x', 'a.y', 'a.z', 'w.x', 'w.y', 'w.z', 
                'q.w', 'q.x', 'q.y', 'q.z',
                'lf.theta', 'lf.beta', 'lf.theta_d', 'lf.beta_d',
                'rf.theta', 'rf.beta', 'rf.theta_d', 'rf.beta_d',
                'rh.theta', 'rh.beta', 'rh.theta_d', 'rh.beta_d',
                'lh.theta', 'lh.beta', 'lh.theta_d', 'lh.beta_d',
                'lf.contact', 'rf.contact', 'rh.contact', 'lh.contact',
                'lf.dist', 'rf.dist', 'rh.dist', 'lh.dist']
data = np.concatenate((p, v, a, w, q,
               lf_theta.reshape(size, 1), lf_beta.reshape(size, 1), lf_theta_d.reshape(size, 1), lf_beta_d.reshape(size, 1), 
               rf_theta.reshape(size, 1), rf_beta.reshape(size, 1), rf_theta_d.reshape(size, 1), rf_beta_d.reshape(size, 1), 
               rh_theta.reshape(size, 1), rh_beta.reshape(size, 1), rh_theta_d.reshape(size, 1), rh_beta_d.reshape(size, 1), 
               lh_theta.reshape(size, 1), lh_beta.reshape(size, 1), lh_theta_d.reshape(size, 1), lh_beta_d.reshape(size, 1), 
               contacts, dst), axis=1)
pd.DataFrame(data[1: -1], columns=columns_name).to_csv('data/'+'data'+filename+'.csv', index=False)
